In [6]:
from __future__ import division, print_function, absolute_import

import os
from decimal import Decimal
from traceback import print_exc

import pandas as pd
import pandas_profiling

# you really want to be efficient about RAM, so user iter and itertools
# from itertools import izip
from twip.constant import DATA_PATH

import matplotlib
%matplotlib inline

In [7]:
# this should load 100k tweets in about a minute
print('Loading tweets (could take a minute or so)...')
df = pd.read_csv(os.path.join(DATA_PATH, 'all_tweets.csv'), encoding='utf-8', engine='python')
# in iPython Notebook print out df.columns to show that many of them contain dots
# rename the columns to be attribute-name friendly
df.columns = [label.replace('.', '_') for label in df.columns]
print('Done.')


Loading tweets (could take a minute or so)...
Done.


In [8]:
# in iPython Notebook, try dropping with lower thresholds, checking column and row count each time
print('The raw table shape is {}'.format(df.shape))
nonnull_rows = 330
nonnull_cols = 50
df = df.dropna(axis=1, thresh=nonnull_rows)
print('After dropping columns with fewer than {} nonnull values, the table shape is {}'.format(nonnull_rows, df.shape))
df = df.dropna(axis=0, thresh=nonnull_cols)
print('After dropping rows with fewer than {} nonnull values, the table shape is {}'.format(nonnull_cols, df.shape))


# in ipython notebook, explore and describe the DataFrame columns
print('Of the {} columns, {} are actually DataFrames'.format(len(df.columns), sum([not isinstance(df[col], pd.Series) for col in df.columns])))
# remove dataframes with only 2 columns and one is the _str of the other:
for col in df.columns:
    if isinstance(df[col], pd.DataFrame):
        print('Column {} is a {}-wide DataFrame'.format(col, len(df[col].columns)))
        if df[col].columns[1] == df[col].columns[0] + '_str':
            print('Column {} looks easy because it has sub-columns {}'.format(col, df[col].columns))
            df[col] = df[col][df[col].columns[1]]
        else:
            try:
                assert(float(df[col].iloc[:, 0].max()) == float(df[col].iloc[:, 1].max()))
                df[col] = df[col].fillna(-1, inplace=False)
                series = pd.Series([int(Decimal(x)) for x in df[col].iloc[:, 1].values]).astype('int64').copy()
                del df[col]
                df[col] = series
                print('Finished converting column {} to type {}({})'.format(col, type(df[col]), df[col].dtype))
            except:
                print_exc()

print('Of the {} columns, {} are still DataFrames after trying to convert both columns to long integers'.format(
    len(df.columns), sum([not isinstance(df[col], pd.Series) for col in df.columns])))

The raw table shape is (99535, 350)
After dropping columns with fewer than 330 nonnull values, the table shape is (99535, 246)
After dropping rows with fewer than 50 nonnull values, the table shape is (98405, 246)
Of the 246 columns, 8 are actually DataFrames
Column quoted_status_id is a 2-wide DataFrame
Finished converting column quoted_status_id to type <class 'pandas.core.series.Series'>(float64)
Column quoted_status_id_str is a 2-wide DataFrame
Finished converting column quoted_status_id_str to type <class 'pandas.core.series.Series'>(float64)
Column retweeted_status_quoted_status_id is a 2-wide DataFrame
Finished converting column retweeted_status_quoted_status_id to type <class 'pandas.core.series.Series'>(float64)
Column retweeted_status_quoted_status_id_str is a 2-wide DataFrame
Finished converting column retweeted_status_quoted_status_id_str to type <class 'pandas.core.series.Series'>(float64)
Of the 242 columns, 0 are still DataFrames after trying to convert both columns to l

In [9]:
print('df.describe() stats:')
desc = df.describe()
for col, stats in desc.T.iterrows():
    print('')
    print('{} ({})'.format(col, df[col].dtype if isinstance(df[col], pd.Series) else type(df[col])))
    print(stats)

df.describe() stats:


/home/hobs/.virtualenvs/twip/local/lib/python2.7/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)



quoted_status_retweet_count (float64)
count      754.000000
mean       225.071618
std       1466.800347
min          0.000000
25%               NaN
50%               NaN
75%               NaN
max      23318.000000
Name: quoted_status_retweet_count, dtype: float64

quoted_status_user_favourites_count (float64)
count       754.000000
mean       8663.230769
std       24785.825849
min           0.000000
25%                NaN
50%                NaN
75%                NaN
max      269482.000000
Name: quoted_status_user_favourites_count, dtype: float64

quoted_status_user_followers_count (float64)
count    7.540000e+02
mean     2.342662e+05
std      1.687835e+06
min      1.200000e+01
25%               NaN
50%               NaN
75%               NaN
max      3.850663e+07
Name: quoted_status_user_followers_count, dtype: float64

quoted_status_user_friends_count (float64)
count       754.000000
mean       3775.824934
std       14528.856528
min           0.000000
25%                NaN
50%     

In [10]:
# this takes a few minutes
print('Using pandas_profiling to generate more detailed stats, including correlation between columns, skew etc')
# pandas_profiling.ProfileReport raises Tkinter exceptions before it can produce any output,
#  at least describe produces a dataframe of stats
report = pandas_profiling.describe(df)
print(report['table'])

print('')
for col, stats in report['variables'].iterrows():
    print('')
    print(col)
    # print('{} ({})'.format(col, df[col].dtype if isinstance(df[col], pd.Series) else type(df[col])))
    print(stats)

# and if you thought that was bad, try printing out all the report['freq'] dict of histograms

Trying to compute a ProfileReport, including correlation between columns, skew etc
{'total_missing': 0.63156452411853847, 'UNIQUE': 0, 'CONST': 3, 'nvar': 243, 'REJECTED': 15, 'n': 98405, 'NUM': 28, 'CORR': 12, 'memsize': '182.4 MiB', 'DATE': 0, 'CAT': 200, 'recordsize': '1.9 KiB', 'n_duplicates': 0}


In [12]:
profile = pandas_profiling.ProfileReport(df)

UnicodeEncodeError: 'ascii' codec can't encode character u'\u2026' in position 213: ordinal not in range(128)